# DB2-Salesforce connector: Citation information updates

In [20]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Record_ID__c'
object_id = 'nanoHUB_citations__c'

# login parameters to be handled by Papermill
# login credentials



    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [21]:
import sys
import os
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import datetime

## Obtain tool information from DB2 

In [22]:
# Basic tool information
sql_query = "select *, c.id as c_id from jos_citations as c \
            left join jos_citations_types as t \
            on c.type = t.id \
            where uid=2143"

# display
print(sql_query)

select *, c.id as c_id from jos_citations as c             left join jos_citations_types as t             on c.type = t.id             where uid=2143


In [23]:
# connect with DB2
import sqlalchemy as sql
from nanoHUB.helpers import ConnectionFactory

nanohub_db = ConnectionFactory().new_for_db('nanohub')

df = pd.read_sql_query(sql_query, nanohub_db)

dsds
2021-06-11 08:14:34,447 - [INFO] nanoHUB.core_containers [connection.get_connection_for:71]: Started SSH Tunnel with db2.nanohub.org
2021-06-11 08:14:34,474 - [DEBUG] paramiko.transport [transport._log:1819]: starting thread (client mode): 0x10061580
2021-06-11 08:14:34,475 - [DEBUG] paramiko.transport [transport._log:1819]: Local version/idstring: SSH-2.0-paramiko_2.7.2
2021-06-11 08:14:34,506 - [DEBUG] paramiko.transport [transport._log:1819]: Remote version/idstring: SSH-2.0-OpenSSH_7.4
2021-06-11 08:14:34,507 - [INFO] paramiko.transport [transport._log:1819]: Connected (version 2.0, client OpenSSH_7.4)
2021-06-11 08:14:34,532 - [DEBUG] paramiko.transport [transport._log:1819]: kex algos:['curve25519-sha256', 'curve25519-sha256@libssh.org', 'ecdh-sha2-nistp256', 'ecdh-sha2-nistp384', 'ecdh-sha2-nistp521', 'diffie-hellman-group-exchange-sha256', 'diffie-hellman-group16-sha512', 'diffie-hellman-group18-sha512', 'diffie-hellman-group-exchange-sha1', 'diffie-hellman-group14-sha256'

In [24]:
# convert and clean some columns, so we can, for example, sort values
df['date_publish'] = pd.to_datetime(df['date_publish'], errors='coerce') 
df['created'] = pd.to_datetime(df['created'], errors='coerce') 
df['date_submit'] = pd.to_datetime(df['date_submit'], errors='coerce') 
df['date_accept'] = pd.to_datetime(df['date_accept'], errors='coerce') 

In [25]:
# display
display(df.head(1))
display('size='+str(df.shape))

,id,uid,affiliated,fundedby,created,address,author,booktitle,chapter,cite,...,custom2,custom3,custom4,pdf_url,id,type,type_title,type_desc,type_export,c_id
0,1816,2143,0,-1,NaT,,Demetrio Logoteta{{82529}};Gianluca Fiori{{981...,,,logoteta2014graphene,...,None,None,None,https://nanohub.org/nanoHUBCitations/citmanage...,2,journal,Journal,None,None,1816


'size=(2507, 68)'

## Match data with Salesforce format

In [26]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here

df_sf['Record_ID__c']      = df['c_id']
df_sf['URL__c']            = df['url']
df_sf['NCN_Affiliated__c'] = df['affiliated'].apply(lambda x: 1 if x==1 else 0)
df_sf['Author_List__c']    = df['author']
df_sf['Journal__c']        = df['journal']
df_sf['Year__c']           = df['year']
df_sf['Type__c']           = df['ref_type']
df_sf['Notes__c']          = df['notes']
df_sf['Name']              = df['title'].apply(lambda x: x[:70])
df_sf['Full_title__c']     = df['title']
df_sf['Format__c']         = df['type_title']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Record_ID__c,1816,330
URL__c,http://scholar.google.com/scholar?hl=en&q=http...,http://papers.ssrn.com/sol3/papers.cfm?abstrac...
NCN_Affiliated__c,0,0
Author_List__c,Demetrio Logoteta{{82529}};Gianluca Fiori{{981...,Shiferaw Gurmu;Grant Black;Paula Stephan
Journal__c,Scientific Reports,Economic Inquiry
Year__c,2014,2007
Type__c,R,R
Notes__c,,URL link for doc is broken. I replaced with vi...
Name,Graphene-based Lateral Heterostructure Transis...,The Knowledge Production Function for Universi...
Full_title__c,Graphene-based Lateral Heterostructure Transis...,The Knowledge Production Function for Universi...


## To Salesforce Sales Cloud CRM

In [27]:
salesforce = ConnectionFactory().new_for_salesforce()
db_s = salesforce
# create DB2 to Salesforce API object

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

2021-06-11 08:14:35,468 - [DEBUG] urllib3.connectionpool [connectionpool._new_conn:971]: Starting new HTTPS connection (1): login.salesforce.com:443
2021-06-11 08:14:35,797 - [DEBUG] urllib3.connectionpool [connectionpool._make_request:452]: https://login.salesforce.com:443 "POST /services/oauth2/token?grant_type=password&client_id=3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG&client_secret=D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764&username=wang2506%40purdue.edu&password=npass2021HDfJLhBGDx11xWCKlEbHQKhF HTTP/1.1" 200 None
Obtained Salesforce access token ...... True


In [28]:
# send data to Salesforce
db_s.send_data(df_sf)

2021-06-11 08:14:35,916 - [DEBUG] urllib3.connectionpool [connectionpool._new_conn:971]: Starting new HTTPS connection (1): na172.salesforce.com:443
2021-06-11 08:14:36,214 - [DEBUG] urllib3.connectionpool [connectionpool._make_request:452]: https://na172.salesforce.com:443 "POST /services/data/v47.0/jobs/ingest/ HTTP/1.1" 200 None
[Success] Bulk job creation successful. Job ID = 7505w00000Ww38WAAR
hello
2021-06-11 08:14:36,249 - [DEBUG] urllib3.connectionpool [connectionpool._new_conn:971]: Starting new HTTPS connection (1): na172.salesforce.com:443
2021-06-11 08:14:38,784 - [DEBUG] urllib3.connectionpool [connectionpool._make_request:452]: https://na172.salesforce.com:443 "PUT /services/data/v47.0/jobs/ingest/7505w00000Ww38WAAR/batches/ HTTP/1.1" 201 None
[Success] CSV upload successful. Job ID = 7505w00000Ww38WAAR
2021-06-11 08:14:38,790 - [DEBUG] urllib3.connectionpool [connectionpool._new_conn:971]: Starting new HTTPS connection (1): na172.salesforce.com:443
2021-06-11 08:14:39,03

In [29]:
# check status
db_s.check_bulk_status()

2021-06-11 08:14:39,049 - [DEBUG] urllib3.connectionpool [connectionpool._new_conn:971]: Starting new HTTPS connection (1): na172.salesforce.com:443
2021-06-11 08:14:39,417 - [DEBUG] urllib3.connectionpool [connectionpool._make_request:452]: https://na172.salesforce.com:443 "GET /services/data/v47.0/jobs/ingest/7505w00000Ww38WAAR HTTP/1.1" 200 None


{'id': '7505w00000Ww38WAAR',
 'operation': 'upsert',
 'object': 'nanoHUB_citations__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-06-11T12:14:36.000+0000',
 'systemModstamp': '2021-06-11T12:14:39.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Record_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [30]:
from pprint import pprint
pprint(db_s.check_bulk_failed_results())

2021-06-11 08:14:39,430 - [DEBUG] urllib3.connectionpool [connectionpool._new_conn:971]: Starting new HTTPS connection (1): na172.salesforce.com:443
2021-06-11 08:14:39,642 - [DEBUG] urllib3.connectionpool [connectionpool._make_request:452]: https://na172.salesforce.com:443 "GET /services/data/v47.0/jobs/ingest/7505w00000Ww38WAAR/failedResults/ HTTP/1.1" 204 0
''
